# Training YOLOv5n
### CS5330
### Talal Siddiqui and Zeyu Wang
### 12/10/2024
This scripts YOLOv5n on a Google Colab instance.

First lets mount GDrive.

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

import os
os.mkdir("raw_data")

In [2]:
os.listdir("gdrive/MyDrive/CS5330/lesion_data/images")

['Images_png_01.zip',
 'Images_png_02.zip',
 'Images_png_03.zip',
 'Images_png_04.zip',
 'Images_png_05.zip',
 'Images_png_06.zip',
 'Images_png_07.zip',
 'Images_png_08.zip',
 'Images_png_09.zip',
 'Images_png_10.zip']

## Unzipping the data into our runtime
Now we unzip all a subset of data from the DeepLesion dataset into our runtime

In [3]:
import os
import zipfile

gdrive_data_dir = "gdrive/MyDrive/CS5330/lesion_data/"
gdrive_images_dir = gdrive_data_dir + "images/"
raw_data_folder = "raw_data/"

if not os.path.exists(raw_data_folder):
  os.mkdir(raw_data_folder)

for image_zip in os.listdir(gdrive_images_dir):
  zip_path = gdrive_images_dir + image_zip
  print("Unzipping", zip_path)

  with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(raw_data_folder)

Unzipping gdrive/MyDrive/CS5330/lesion_data/images/Images_png_01.zip
Unzipping gdrive/MyDrive/CS5330/lesion_data/images/Images_png_02.zip
Unzipping gdrive/MyDrive/CS5330/lesion_data/images/Images_png_03.zip
Unzipping gdrive/MyDrive/CS5330/lesion_data/images/Images_png_04.zip
Unzipping gdrive/MyDrive/CS5330/lesion_data/images/Images_png_05.zip
Unzipping gdrive/MyDrive/CS5330/lesion_data/images/Images_png_06.zip
Unzipping gdrive/MyDrive/CS5330/lesion_data/images/Images_png_07.zip
Unzipping gdrive/MyDrive/CS5330/lesion_data/images/Images_png_08.zip
Unzipping gdrive/MyDrive/CS5330/lesion_data/images/Images_png_09.zip
Unzipping gdrive/MyDrive/CS5330/lesion_data/images/Images_png_10.zip


### Installing ultralytics
This package has the implementation of YOLOv5n we will use.

In [4]:
!pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 898.8/898.8 kB 21.9 MB/s eta 0:00:00


## Load into annotations
Lets take a look as to how the annotation CSV looks like.

In [5]:
import pandas as pd

df = pd.read_csv(gdrive_data_dir + "DL_info.csv")
df

,File_name,Patient_index,Study_index,Series_ID,Key_slice_index,Measurement_coordinates,Bounding_boxes,Lesion_diameters_Pixel_,Normalized_lesion_location,Coarse_lesion_type,Possibly_noisy,Slice_range,Spacing_mm_px_,Image_size,DICOM_windows,Patient_gender,Patient_age,Train_Val_Test
0,000001_01_01_109.png,1,1,1,109,"233.537, 95.0204, 234.057, 106.977, 231.169, 1...","226.169, 90.0204, 241.252, 111.977","11.9677, 5.10387","0.44666, 0.283794, 0.434454",3,0,"103, 115","0.488281, 0.488281, 5","512, 512","-175, 275",F,62.0,3
1,000001_02_01_014.png,1,2,1,14,"224.826, 289.296, 224.016, 305.294, 222.396, 2...","217.396, 284.296, 233.978, 310.294","16.019, 6.61971","0.431015, 0.485238, 0.340745",3,0,"8, 23","0.314453, 0.314453, 5","512, 512","-175, 275",F,72.0,3
2,000001_02_01_017.png,1,2,1,17,"272.323, 320.763, 246.522, 263.371, 234.412, 3...","229.412, 258.371, 285.221, 325.763","62.9245, 48.9929","0.492691, 0.503106, 0.351754",3,0,"8, 23","0.314453, 0.314453, 5","512, 512","-175, 275",F,72.0,3
3,000001_03_01_088.png,1,3,1,88,"257.759, 157.618, 260.018, 133.524, 251.735, 1...","246.735, 128.524, 270.288, 162.618","24.1998, 13.6123","0.498999, 0.278924, 0.452792",3,0,"58, 118","0.732422, 0.732422, 1","512, 512","-175, 275",F,73.0,3
4,000001_04_01_017.png,1,4,1,17,"304.019, 230.585, 292.217, 211.789, 304.456, 2...","287.217, 206.789, 309.456, 235.585","22.1937, 9.8065","0.572678, 0.42336, 0.445674",3,0,"11, 23","0.666016, 0.666016, 5","512, 512","-175, 275",F,73.0,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32730,004458_01_01_059.png,4458,1,1,59,"395.737, 284.142, 385.026, 310.221, 402.198, 3...","376.707, 279.142, 407.198, 315.221","28.1937, 22.1522","0.862386, 0.522347, 0.623233",-1,0,"40, 85","0.857422, 0.857422, 5","512, 512","-175, 275",F,66.0,1
32731,004458_01_01_049.png,4458,1,1,49,"411.677, 296.13, 380.941, 321.278, 404.527, 32...","375.941, 291.13, 416.677, 328.028","39.7138, 30.688","0.845748, 0.557232, 0.570726",-1,0,"40, 85","0.857422, 0.857422, 5","512, 512","-175, 275",F,66.0,1
32732,004458_01_01_047.png,4458,1,1,47,"204.055, 307.341, 170.022, 316.047, 180.311, 3...","165.022, 297.592, 209.055, 329.357","35.1289, 22.4662","0.339313, 0.588705, 0.56127",-1,0,"40, 85","0.857422, 0.857422, 5","512, 512","-175, 275",F,66.0,1
32733,004458_01_01_102.png,4458,1,1,102,"276.457, 361.216, 220.572, 354.23, 247.117, 35...","215.572, 347.367, 281.457, 373.201","56.3199, 15.9573","0.477197, 0.730823, 0.813426",-1,0,"96, 108","0.857422, 0.857422, 5","512, 512","-175, 275",F,66.0,1


In [6]:
def get_image_path(file_name):
  images_dir = "raw_data/Images_png/"

  chunks = file_name.split("_")
  image_dir = images_dir + "_".join(chunks[0:3])
  image_name = chunks[-1]
  image_path = image_dir + "/" + image_name

  return image_path

In [7]:
get_image_path("000001_01_01_011.png")

'raw_data/Images_png/000001_01_01/011.png'

### Pre-processing and filtering
We have to filter out rows in the annotation CSV that refer to images not in our subset and also transform each bounding box annotation from x1, y1, x2, y2 format to normalized x_center, y_center, width, and height format

In [8]:
import os
from collections import defaultdict

img_width = 512
img_height = 512

num_decimals = 6

processed_df = defaultdict(list)

for idx, row in df[["File_name", "Bounding_boxes", "Coarse_lesion_type"]].iterrows():
  image_path = get_image_path(row["File_name"])

  if not os.path.exists(image_path):
    break

  bbox = row["Bounding_boxes"]
  lesion_type = row["Coarse_lesion_type"]

  x1, y1, x2, y2 = [float(coord) for coord in bbox.split(",")]

  width = x2 - x1
  x_center = x1 + (width / 2)
  # Normalize these
  width = round(width / img_width, num_decimals)
  x_center = round(x_center / img_width, num_decimals)

  height = y2 - y1
  y_center = y1 + (height / 2)
  # Normalize these
  height = round(height / img_height, num_decimals)
  y_center = round(y_center / img_height, num_decimals)


  fixed_lesion_type = 0 if lesion_type == -1 else lesion_type

  processed_df[row["File_name"]].append([fixed_lesion_type, x_center, y_center, width, height])

In [9]:
processed_df_list = list(processed_df.items())

In [10]:
print("Num images = ", len(processed_df_list))

Num images =  6988


In [11]:
processed_df_list[:10]

[('000001_01_01_109.png', [[3, 0.456466, 0.197263, 0.029459, 0.042884]]),
 ('000001_02_01_014.png', [[3, 0.440795, 0.580654, 0.032387, 0.050777]]),
 ('000001_02_01_017.png', [[3, 0.502571, 0.570443, 0.109002, 0.131625]]),
 ('000001_03_01_088.png', [[3, 0.504905, 0.284318, 0.046002, 0.06659]]),
 ('000001_04_01_017.png', [[3, 0.582688, 0.432006, 0.043436, 0.056242]]),
 ('000002_01_01_162.png', [[2, 0.463459, 0.517404, 0.03152, 0.033918]]),
 ('000002_01_01_176.png', [[2, 0.524312, 0.372269, 0.048088, 0.038826]]),
 ('000002_02_01_077.png', [[2, 0.458464, 0.425654, 0.043963, 0.041172]]),
 ('000002_02_01_050.png', [[2, 0.593257, 0.509947, 0.058076, 0.050617]]),
 ('000002_02_01_065.png', [[2, 0.553994, 0.398123, 0.041945, 0.039906]])]

In [12]:
from collections import Counter

Counter([lesion[0] for row in processed_df_list for lesion in row[1]])

Counter({3: 396,
         2: 504,
         0: 5294,
         7: 141,
         5: 298,
         6: 90,
         1: 47,
         8: 186,
         4: 192})

### Train-test split
Split our data into train and test/validation subsets.

In [13]:
train_pct = 0.80

num_rows = len(processed_df_list)
num_train = int(len(processed_df_list) * train_pct)

train_df = processed_df_list[:num_train]
test_df = processed_df_list[num_train:]

print("# train =", num_train)
print("# test =", num_rows - num_train)

# train = 5590
# test = 1398


### Move images into proper folder structure
The implementation of YOLOv5n we're using expects a certain directory strutcture for the data you give it. Train and test must have their own folders, and each of them has "images" and "labels" folder where each image in "images/" has a corresponding file in "labels/" with the same name but a ".txt" extension instead.

In [14]:
import os
import shutil

datasets_root_dir = "datasets/"
data_dir = datasets_root_dir + "data/"

train_dir = data_dir + "train/"
train_imgs_dir = train_dir + "images"
train_labels_dir = train_dir + "labels"

test_dir = data_dir + "test/"
test_imgs_dir = test_dir + "images"
test_labels_dir = test_dir + "labels"


for dir in [datasets_root_dir, data_dir,
            train_dir, train_imgs_dir, train_labels_dir,
            test_dir, test_imgs_dir, test_labels_dir]:
  if not os.path.exists(dir):
    os.mkdir(dir)

def copy_data_in_yolo_fmt(target_df, imgs_dir, labels_dir):
  for idx, row in enumerate(target_df):
    image_str, lesions = row

    # image_str, lesion_type, x_center, y_center, width, height = row

    if idx % 500 == 0:
      print("Copying image",  idx)

    image_path = get_image_path(image_str)

    img_dst_path = imgs_dir + "/" + image_str

    text_file_name = image_str.split(".")[0] + ".txt"
    text_file_path =  labels_dir + "/" + text_file_name


    # Copy image
    shutil.copyfile(image_path, img_dst_path)

    # Write text file
    with open(text_file_path, mode="w") as text_file:
      for idx, lesion_row in enumerate(lesions):
        # print(lesion_row)
        lesion_type, x_center, y_center, width, height = lesion_row
        obj_str = f"{lesion_type} {x_center} {y_center} {width} {height}"

        if idx != len(lesions) - 1:
          obj_str += "\n"

        text_file.write(obj_str)

In [15]:
print("Copying train data")
copy_data_in_yolo_fmt(train_df, train_imgs_dir, train_labels_dir)

print("Copying test data")
copy_data_in_yolo_fmt(test_df, test_imgs_dir, test_labels_dir)

Copying train data
Copying image 0
Copying image 500
Copying image 1000
Copying image 1500
Copying image 2000
Copying image 2500
Copying image 3000
Copying image 3500
Copying image 4000
Copying image 4500
Copying image 5000
Copying image 5500
Copying test data
Copying image 0
Copying image 500
Copying image 1000


### Create `data.yml` file
We also have to create a `data.yml` file that stores the paths to the dataset folder, the train and test subfolders, and the classes present in the dataset.

In [16]:
data_yml = """path: data
train: train/images
val: test/images
names:
    0: other
    1: bone
    2: abdomen
    3: mediastinum
    4: liver
    5: lung
    6: kidney
    7: soft_tissue
    8: pelvis
"""

with open("data.yml", mode="w") as data_yml_file:
  data_yml_file.write(data_yml)

#### Clear memory
Useful when doing re-runs and when the VRAM is completely full.

In [27]:
import gc
import torch
torch.cuda.empty_cache()
gc.collect()

65561

### Train!
Lets train our YOLOv5n model

In [ ]:
from ultralytics import YOLO

# Load a COCO-pretrained YOLOv5n model
model = YOLO("yolov5n.pt")

# Display model information (optional)
model.info()

# Prev - epochs=70, batch=150

results = model.train(data="data.yml", epochs=160, imgsz=512, batch=150, patience=120)

PRO TIP 💡 Replace 'model=yolov5n.pt' with new 'model=yolov5nu.pt'.
YOLOv5 'u' models are trained with https://github.com/ultralytics/ultralytics and feature improved performance vs standard YOLOv5 models trained with https://github.com/ultralytics/yolov5.

YOLOv5n summary: 262 layers, 2,654,816 parameters, 0 gradients, 7.8 GFLOPs
Ultralytics 8.3.47 🚀 Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=detect, mode=train, model=yolov5n.pt, data=data.yml, epochs=160, time=None, patience=120, batch=150, imgsz=512, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train5, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, ma

train: Scanning /content/datasets/data/train/labels.cache... 5590 images, 0 backgrounds, 6 corrupt: 100%|██████████| 5590/5590 [00:00<?, ?it/s]

train: WARNING ⚠️ /content/datasets/data/train/images/000027_01_01_113.png: ignoring corrupt image/label: non-normalized or out of bounds coordinates [      1.035]
train: WARNING ⚠️ /content/datasets/data/train/images/000027_02_01_105.png: ignoring corrupt image/label: non-normalized or out of bounds coordinates [     1.0744]
train: WARNING ⚠️ /content/datasets/data/train/images/000084_04_02_141.png: ignoring corrupt image/label: non-normalized or out of bounds coordinates [     1.0515]
train: WARNING ⚠️ /content/datasets/data/train/images/000332_07_02_262.png: ignoring corrupt image/label: non-normalized or out of bounds coordinates [     1.0677]
train: WARNING ⚠️ /content/datasets/data/train/images/000343_01_01_152.png: ignoring corrupt image/label: non-normalized or out of bounds coordinates [     1.0541]
train: WARNING ⚠️ /content/datasets/data/train/images/000417_03_01_125.png: ignoring corrupt image/label: non-normalized or out of bounds coordinates [     1.0681]


albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, num_output_channels=3, method='weighted_average'), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))


/usr/local/lib/python3.10/dist-packages/albumentations/__init__.py:24: UserWarning: A new version of Albumentations is available: 1.4.22 (you have 1.4.20). Upgrade using: pip install -U albumentations. To disable automatic update checks, set the environment variable NO_ALBUMENTATIONS_UPDATE to 1.
  check_for_updates()
val: Scanning /content/datasets/data/test/labels.cache... 1398 images, 0 backgrounds, 1 corrupt: 100%|██████████| 1398/1398 [00:00<?, ?it/s]

...


      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


    111/160      12.1G      1.496      1.381      1.059         48        512: 100%|██████████| 38/38 [01:37<00:00,  2.56s/it]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 5/5 [00:18<00:00,  3.72s/it]

                   all       1397       1434      0.124       0.15      0.095     0.0532



      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


    112/160      12.1G      1.495      1.402      1.058         46        512: 100%|██████████| 38/38 [01:36<00:00,  2.55s/it]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95):   0%|          | 0/5 [00:00<?, ?it/s]

In [ ]:
# Have to do this or else you get a weird encoding error when using the zip command

import locale

locale.getpreferredencoding = lambda: "UTF-8"

## Zip and download
The Ultralytics package stores metrics and visualizations of the models performance in a "runs/" folder so all we have to do is zip it and use the Google Colab API to automatically download it.


This is extremely useful when running the model overnight and making sure that the results and weights are downloaded right after the model is done training.

In [ ]:
! zip -r runs.zip runs/

In [ ]:
from google.colab import files
files.download("runs.zip")